# Line RT: 1D homogeneous,
# velocity gradient

## 0) Setup

In [1]:
# Define Magritte folders

MagritteSetupFolder = '/home/frederik/Dropbox/Astro/Magritte/modules/setup/'
ProjectFolder       = '/home/frederik/MagritteProjects/Lines_1D_LTE/'

In [3]:
# Import tools and libraries

import numpy as np

from bokeh.plotting import figure, show, gridplot
from bokeh.palettes import cividis
from bokeh.io       import output_notebook
output_notebook()

from os   import getcwd
from glob import glob
from sys  import path
path.insert(0, MagritteSetupFolder)

# Import from MagritteSetupFolder
from lineData import LineData, planck, relativeDifference
from spheres  import nRays
from setup    import setupMagritte
from model    import model
from build    import compile, execute

Loading BokehJS ...

## 1) Define model

Assume a 1D model (i.e. 1 ray) with constant temperature, density and abundances and a linear velocity field, yielding a constant velocity gradient

\begin{align}
T(x)     \ &= \ cte, \\
\rho(x)  \ &= \ cte, \\
n_{i}(x) \ &= \ cte, \\
v(x)     \ &= \ v_{\max} \ x \ / \ L,
\end{align}

where $L$ is the total length of the model and $v_{\max}$ is the maximum speed reached in the model. (Numerical values given below.) We write it in this way to ensure that velocities in the model are always much smaller than $c$, since our equations for Doppler shift are not valid otherwise.

There is CMB radiation incoming on both ends of the ray as boundary condition.

In [4]:
model = model (dim=1)

model.ncells = 20

dens = 1.0E+12   # [m^-3]
abun = 1.0E+4    # [m^-3]
temp = 1.0E+02   # [K]
dx   = 1.0E5     # [m]
bx   = 1.3E0
dT   = 0.0E0     # [K]
dv   = 5.0E2    # [m/s]

model.density     = [dens     for _ in range(model.ncells)]
model.abundance   = [abun     for _ in range(model.ncells)]
model.temperature = [temp     for _ in range(model.ncells)]

#model.x = [dx*i  for i in range(model.ncells)]
model.x = [dx*i  for i in range(model.ncells)]
model.y = [0.0   for _ in range(model.ncells)]
model.z = [0.0   for _ in range(model.ncells)]

model.vx = [dv*i for i in range(model.ncells)]
model.vy = [0.0  for _ in range(model.ncells)]
model.vz = [0.0  for _ in range(model.ncells)]

model.boundary = [0, model.ncells-1]

model.defineRays (nrays=nRays(nsides=1))

model.getNeighborLists ()

# Write new model data
model.writeInput (ProjectFolder + 'Magritte_files/')

# Run setup
setupMagritte (projectFolder = ProjectFolder, runName = '')

'/home/frederik/MagritteProjects/Lines_1D_LTE/io/19-01-03_19:40:58/'

In [5]:
# Plot model

plot_model_1 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_1.line (model.x, model.density)
plot_model_1.xaxis.axis_label = "x [m]"
plot_model_1.yaxis.axis_label = "density [m^-3]"

plot_model_2 = figure (plot_width=400, plot_height=250, y_axis_type='log')
plot_model_2.line (model.x, model.abundance)
plot_model_2.xaxis.axis_label = "x [m]"
plot_model_2.yaxis.axis_label = "abundance [xm^-3]"

plot_model_3 = figure(plot_width=400, plot_height=250)
plot_model_3.line (model.x, model.temperature)
plot_model_3.xaxis.axis_label = "x [m]"
plot_model_3.yaxis.axis_label = "temperature [K]"

plot_model_4 = figure(plot_width=400, plot_height=250)
plot_model_4.line (model.x, model.vx)
plot_model_4.xaxis.axis_label = "x [m]"
plot_model_4.yaxis.axis_label = "velocity_x [m^-3]"

plot_model = gridplot ([[plot_model_1, plot_model_2],[plot_model_3, plot_model_4]])

show (plot_model)

### Compile Magrittte and Run Model 

In [6]:
compile (MagritteSetupFolder, ProjectFolder)
execute (MagritteSetupFolder, ProjectFolder)

-- Found OpenMP_C: -fopenmp
-- Found OpenMP_CXX: -fopenmp
-- Found OpenMP_C: -fopenmp
-- Found OpenMP_CXX: -fopenmp
-- Found OpenMP_C: -fopenmp
-- Found OpenMP_CXX: -fopenmp
-- Found OpenMP_C: -fopenmp
-- Found OpenMP_CXX: -fopenmp
-- /opt/intel/compilers_and_libraries_2017.4.196/linux/mpi/intel64/include
-- Found OpenMP_C: -fopenmp
-- Found OpenMP_CXX: -fopenmp
-- Configuring done
-- Generating done
-- Build files have been written to: /home/frederik/MagritteProjects/Lines_1D_LTE/build
[ 14%] Built target _TEST_RT_MAIN
[ 14%] Built target _TEST_MAIN
[ 14%] Built target _HEALPix
Scanning dependencies of target _RADIATIVE_TRANSFER
[ 16%] Building CXX object src/RadiativeTransfer/src/CMakeFiles/_RADIATIVE_TRANSFER.dir/frequencies.cpp.o
[ 19%] Building CXX object src/RadiativeTransfer/src/CMakeFiles/_RADIATIVE_TRANSFER.dir/lines.cpp.o
[ 21%] Building CXX object src/RadiativeTransfer/src/CMakeFiles/_RADIATIVE_TRANSFER.dir/raypair.cpp.o
[ 23%] Building CXX object src/RadiativeTransfer/src/C

## 2) Get Magritte output

In [7]:
# Get Magritte data

ioFolders = glob(ProjectFolder + 'io/*/')
ioFolders.sort()

inputFolders  = [folder +  'input/' for folder in ioFolders]
outputFolders = [folder + 'output/' for folder in ioFolders]

lastOutput = outputFolders[-1]
lastInput  =  inputFolders[-1]

pops_files = glob(lastOutput + 'populations_0*.txt')
Jeff_files = glob(lastOutput + 'Jeff_0*.txt')
J_files    = glob(lastOutput + 'J_*.txt')
G_files    = glob(lastOutput + 'G_*.txt')
nu_files   = glob(lastOutput + 'frequencies_nu*.txt')
lnr_files  = glob(lastOutput + 'frequencies_line_nr*.txt')
eta_files  = glob(lastOutput + 'eta_0*.txt')
chi_files  = glob(lastOutput + 'chi_0*.txt')

pops_files.sort()
Jeff_files.sort()
eta_files.sort()
chi_files.sort()

pops_data = [np.loadtxt(fileName) for fileName in pops_files]
Jeff_data = [np.loadtxt(fileName) for fileName in Jeff_files]
J_data    = [np.loadtxt(fileName) for fileName in J_files]
G_data    = [np.loadtxt(fileName) for fileName in G_files]
nu_data   = [np.loadtxt(fileName) for fileName in nu_files]
lnr_data  = [np.loadtxt(fileName) for fileName in lnr_files]
eta_data  = [np.loadtxt(fileName) for fileName in eta_files]
chi_data  = [np.loadtxt(fileName) for fileName in chi_files]

# Import linedata
lineData = LineData (ProjectFolder + '/Magritte_files/linedata/hco+.dat')

### 2.1) Plot output

In [8]:
# Plot functions

def color(s):
    ns = int((s_max-s_min) / s_step + 1)
    es = int((s    -s_min) / s_step)
    return cividis(ns)[es]

def legend(s):
    return f'{s}'

def bokeh_log_plot(title, x, y, xlabel, ylabel):
    return

In [9]:
s_min  = 0
s_max  = model.ncells
s_step = 1

In [10]:
# Level populations

plot = figure (title='Level populations', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = range(lineData.nlev)
    y = pops_data[0][s]
    plot.line (x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "number of the level"
plot.yaxis.axis_label = "population [m^-3]"
show (plot)

In [11]:
# Mean intensity

plot = figure (title='Total mean intensity', width=700, height=400, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = range(lineData.nrad)
    y = Jeff_data[1][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "number of the transition"
plot.yaxis.axis_label = "mean intensity J [m^-3]"
show(plot)

In [12]:
# Spectrum

plot = figure (title='Spectrum', width=700, height=500, y_axis_type='log')
for s in range(s_min, s_max, s_step):
    x = nu_data[0][s]
    y =  J_data[0][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "frequencies [Hz]"
plot.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot)

In [13]:
# Flux (G) spectrum

plot = figure (title='Spectrum', width=700, height=500)
for s in range(s_min, s_max, s_step):
    x = nu_data[0][s]
    y =  G_data[0][s]
    plot.line(x, y, color=color(s), legend=legend(s))
plot.xaxis.axis_label = "frequencies [Hz]"
plot.yaxis.axis_label = "Mean intensity [W/m^2]"
show(plot)

## 3) Analytical solution

Assuming a constant source function $S_{\nu}(x)=S_{\nu}$ along the ray and boundary condition $B_{\nu}$ on both sides of the ray, the mean intensity is given by

\begin{align}
    J_{\nu}(\tau(x)) \ &= \ S_{\nu} \ + \ \frac{1}{2} \ \left(B_{\nu}-S_{\nu}\right) \ \left[e^{-\tau_{\nu}(x)} + e^{-\tau_{\nu}(L-x)}\right], \\
    G_{\nu}(\tau(x)) \ &= \ \color{white}S_{\color{white}\nu} \ - \ \frac{1}{2} \ \left(B_{\nu}-S_{\nu}\right) \ \left[e^{-\tau_{\nu}(x)} - e^{-\tau_{\nu}(L-x)}\right],
\end{align}

where the optical depth $\tau_{\nu}$ is given by

\begin{equation}
    \tau_{\nu}(\ell) \ = \ \int_{0}^{\ell} \text{d} l \ \chi_{\nu}(l) .
\end{equation}

The frequency dependence of the opacity only comes from the line profile

\begin{equation}
    \chi_{\nu}(x) \ = \ \chi_{ij} \phi_{\nu},
\end{equation}

where we assume a Gaussian profile

\begin{equation}
	\phi_{\nu}^{ij}(x) \ = \ \frac{1}{\sqrt{\pi} \ \delta\nu_{ij}} \ \exp \left[-\left(\frac{\nu-\nu_{ij}} {\delta\nu_{ij}(x)}\right)^{2}\right], \hspace{5mm} \text{where} \hspace{5mm} \delta\nu_{ij}(x) \ = \ \frac{\nu_{ij}}{c} \sqrt{ \frac{2 k_{b} T(x)}{m_{\text{spec}}} \ + \ v_{\text{turb}}^{2}(x)}.
\end{equation}

To account for the velocity gradient, after a slab of length $\ell$, the frequency shifts as

\begin{equation}
    \nu \ \rightarrow \ \left( 1 - \frac{v_{\max} \ell}{c L} \right) \nu.
\end{equation}

Solving the integral for the optical depth then yields

\begin{equation}
  \tau_{\nu}(\ell) \ = \ \frac{\chi L}{\nu} \ \frac{c}{v_{\max}}  \ \frac{1}{2} \left\{ \text{Erf}\left[\frac{\nu-\nu_{ij}}{\delta\nu_{ij}}\right] \ + \ \text{Erf}\left[\frac{v_{\max}}{c} \frac{\nu}{\delta\nu_{ij}}\frac{\ell}{L} - \frac{\nu-\nu_{ij}}{\delta\nu_{ij}}\right] \right\} .
\end{equation}

In [14]:
# Set line
line = 15

In [15]:
# Analytical model

from scipy.special import erf

c     = 2.99792458E+8    # [m/s] speed of light
kb    = 1.38064852E-23   # [J/K] Boltzmann's constant
mp    = 1.6726219E-27    # [kg]  proton mass
T_CMB = 2.7254800        # [K]   CMB temperature
vturb = 0.150E3          # [m/s] turbulent speed

pops       = lineData.LTEpop(temp) * abun
emissivity = lineData.lineEmissivity(pops)
opacity    = lineData.lineOpacity(pops)
source     = emissivity / opacity

def bcd (nu):
    return planck(T_CMB, nu)

S    =  source[line]
eta  = emissivity[line]
chi  = opacity[line]
L    = model.x[-1]
vmax = model.vx[-1]
nuij = lineData.frequency[line]
dnu  = nuij/c * np.sqrt(2.0*kb*temp/mp + vturb**2)

print(f'velocity gradient = {vmax/L}')
print(f'temperature       = {temp}')

def tau(nu, l):
    arg = (nu - nuij) / dnu
    fct = vmax/c * nu/dnu
    return chi*L / (fct*dnu) * 0.5 * (erf(arg) + erf(fct*l/L-arg))

def J(nu, x):
    tau1 = tau(nu, x)
    tau2 = tau(nu, L-x)
    B = bcd (nu)
    return S + 0.5 * (B-S) * (np.exp(-tau1) + np.exp(-tau2))

def G(nu, x):
    tau1 = tau(nu, x)
    tau2 = tau(nu, L-x)
    B = bcd (nu)
    return   - 0.5 * (B-S) * (np.exp(-tau1) - np.exp(-tau2))

def relativeError(a,b):
    return 2.0 * np.abs((a-b)/(a+b))

velocity gradient = 0.005
temperature       = 100.0


In [16]:
# Line

plot_model = figure(title='Line model', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 18    )
    N = int(lnr_data[0][s][line] + 18 + 1)
    # model
    x = nuij + 18 * dnu * np.linspace(-1,1,500)
    y = J(x, model.x[s])
    plot_model.line(x, y, color=color(s))
    # data
    x = nu_data[0][s][M:N]
    y =  J_data[0][s][M:N]
    plot_model.circle(x, y, color=color(s), legend=legend(s))

plot_error = figure(title='Line error', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 18    )
    N = int(lnr_data[0][s][line] + 18 + 1)
    # error
    x = nu_data[0][s][M:N]
    y = relativeError(J(x, model.x[s]), J_data[0][s][M:N])
    plot_error.circle(x, y, color=color(s), legend=legend(s))

plot = gridplot([[plot_model, plot_error]])

show(plot)

In [17]:
# Line model

plot_model = figure(title='Line model', width=400, height=400)
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 18    )
    N = int(lnr_data[0][s][line] + 18 + 1)
    # model
    x = nuij + 18 * dnu * np.linspace(-1,1,500)
    y = G(x, model.x[s])
    plot_model.line(x, y, color=color(s))
    # data
    x = nu_data[0][s][M:N]
    y =  G_data[0][s][M:N]
    plot_model.circle(x, y, color=color(s), legend=legend(s))

plot_error = figure(title='Line error', width=400, height=400, y_axis_type="log")
for s in range(s_min, s_max, s_step):
    M = int(lnr_data[0][s][line] - 18    )
    N = int(lnr_data[0][s][line] + 18 + 1)
    # error
    x = nu_data[0][s][M:N]
    y = relativeError(G(x, model.x[s]), G_data[0][s][M:N])
    plot_error.circle(x, y, color=color(s), legend=legend(s))

plot = gridplot([[plot_model, plot_error]])

show(plot)

In [18]:
from os  import getcwd
from sys import path
path.insert(0, f'{getcwd()}/functions')

from functions import test, optical_depth_m, optical_depth_p, mean_intensity

ModuleNotFoundError: No module named 'functions'

In [16]:
f'{getcwd()}/functions'

'/home/frederik/Dropbox/GitHub/LineRadiativeTransfer/functions'

In [17]:
from functions import test

test(33)

5.0

In [27]:
nu  = nuij*1.000001

zero = model.x[0]
xx   = model.x[5]*2.4565
L    = model.x[-1]

dx = (L-zero) / 10000

print(optical_depth_p (nu, chi, nuij, zero, xx, dx))
print(tau(nu, xx-zero))

print(bcd(nu) + mean_intensity(nu, eta, chi, nuij, zero, xx, L, dx))
print(J(nu, xx))

2.406414061420342e-10
2.4056612847720573e-10
1.099759694596062e-23
1.0996039442958794e-23
